In [10]:
#!python -m pip install -q "qdrant-client[fastembed]>=1.14.2"

In [11]:
from qdrant_client import QdrantClient, models

In [12]:
client = QdrantClient("http://localhost:6333") 

In [13]:
import requests

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [14]:
#documents_raw[1]

In [15]:
from fastembed import TextEmbedding
TextEmbedding.list_supported_models()

[{'model': 'BAAI/bge-base-en',
  'sources': {'hf': 'Qdrant/fast-bge-base-en',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.42,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model': 'BAAI/bge-base-en-v1.5',
  'sources': {'hf': 'qdrant/bge-base-en-v1.5-onnx-q',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.21,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model':

In [16]:
import json

EMBEDDING_DIMENSIONALITY = 512

for model in TextEmbedding.list_supported_models():
    if model["dim"] == EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))


{
  "model": "BAAI/bge-small-zh-v1.5",
  "sources": {
    "hf": "Qdrant/bge-small-zh-v1.5",
    "url": "https://storage.googleapis.com/qdrant-fastembed/fast-bge-small-zh-v1.5.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), Chinese, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.09,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "Qdrant/clip-ViT-B-32-text",
  "sources": {
    "hf": "Qdrant/clip-ViT-B-32-text",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model.onnx",
  "description": "Text embeddings, Multimodal (text&image), English, 77 input tokens truncation, Prefixes for queries/documents: not necessary, 2021 year",
  "license": "mit",
  "size_in_GB": 0.25,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "jinaai/jina-embeddings-v2-small-e

In [17]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [22]:
# Define the collection name
collection_name = "zoomcamp-rag2"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [23]:
#http://localhost:6333/dashboard#/collections

In [24]:
points = []
id = 0

for course in documents_raw:
    for doc in course['documents']:

        point = models.PointStruct(
            id=id,
            vector=models.Document(text=doc['text'], model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
            payload={
                "text": doc['text'],
                "section": doc['section'],
                "course": course['course']
            } #save all needed metadata fields
        )
        points.append(point)

        id += 1


In [25]:
client.upsert(
    collection_name=collection_name,
    points=points
)


Fetching 5 files: 100%|██████████████████████████████████████████████| 5/5 [00:02<00:00,  1.95it/s]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [26]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [27]:
import random

course = random.choice(documents_raw)
course_piece = random.choice(course['documents'])
print(json.dumps(course_piece, indent=2))


{
  "text": "Solution:\nIf you\u2019re using Mage, in the last Data Exporter that writes to Google Cloud Storage use PyArrow to generate the Parquet file with the correct logical type for the datetime columns, otherwise they won't be converted to timestamp when loaded by BigQuery later on.\nimport pyarrow as pa\nimport pyarrow.parquet as pq\nimport os\nif 'data_exporter' not in globals():\nfrom mage_ai.data_preparation.decorators import data_exporter\n# Replace with the location of your service account key JSON file.\nos.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/src/personal-gcp.json'\nbucket_name = \"<YOUR_BUCKET_NAME>\"\nobject_key = 'nyc_taxi_data_2022.parquet'\nwhere = f'{bucket_name}/{object_key}'\n@data_exporter\ndef export_data(data, *args, **kwargs):\ntable = pa.Table.from_pandas(data, preserve_index=False)\ngcs = pa.fs.GcsFileSystem()\npq.write_table(\ntable,\nwhere,\n# Convert integer columns in Epoch milliseconds\n# to Timestamp columns in microseconds ('us') so\n# 

In [28]:
result = search(course_piece['question'])

In [29]:
result

QueryResponse(points=[ScoredPoint(id=212, version=0, score=0.8810406, payload={'text': 'Background:\n`pd.read_parquet`\n`pd.to_datetime`\n`pq.write_to_dataset`\nReference:\nhttps://stackoverflow.com/questions/48314880/are-parquet-file-created-with-pyarrow-vs-pyspark-compatible\nhttps://stackoverflow.com/questions/57798479/editing-parquet-files-with-python-causes-errors-to-datetime-format\nhttps://www.reddit.com/r/bigquery/comments/16aoq0u/parquet_timestamp_to_bq_coming_across_as_int/?share_id=YXqCs5Jl6hQcw-kg6-VgF&utm_content=1&utm_medium=ios_app&utm_name=ioscss&utm_source=share&utm_term=1\nSolution:\nAdd `use_deprecated_int96_timestamps=True` to `pq.write_to_dataset` function, like below\npq.write_to_dataset(\ntable,\nroot_path=root_path,\nfilesystem=gcs,\nuse_deprecated_int96_timestamps=True\n# Write timestamps to INT96 Parquet format\n)', 'section': 'Module 3: Data Warehousing', 'course': 'data-engineering-zoomcamp'}, vector=None, shard_key=None, order_value=None)])

In [30]:
print(f"Question:\n{course_piece['question']}\n")
print("Top Retrieved Answer:\n{}\n".format(result.points[0].payload['text']))
print("Original Answer:\n{}".format(course_piece['text']))


Question:
GCP BQ - Datetime columns in Parquet files created from Pandas show up as integer columns in BigQuery

Top Retrieved Answer:
Background:
`pd.read_parquet`
`pd.to_datetime`
`pq.write_to_dataset`
Reference:
https://stackoverflow.com/questions/48314880/are-parquet-file-created-with-pyarrow-vs-pyspark-compatible
https://stackoverflow.com/questions/57798479/editing-parquet-files-with-python-causes-errors-to-datetime-format
https://www.reddit.com/r/bigquery/comments/16aoq0u/parquet_timestamp_to_bq_coming_across_as_int/?share_id=YXqCs5Jl6hQcw-kg6-VgF&utm_content=1&utm_medium=ios_app&utm_name=ioscss&utm_source=share&utm_term=1
Solution:
Add `use_deprecated_int96_timestamps=True` to `pq.write_to_dataset` function, like below
pq.write_to_dataset(
table,
root_path=root_path,
filesystem=gcs,
use_deprecated_int96_timestamps=True
# Write timestamps to INT96 Parquet format
)

Original Answer:
Solution:
If you’re using Mage, in the last Data Exporter that writes to Google Cloud Storage use P

In [32]:
search("What if I submit homeworks late?")

QueryResponse(points=[ScoredPoint(id=15, version=0, score=0.9072734, payload={'text': 'No, late submissions are not allowed. But if the form is still not closed and it’s after the due date, you can still submit the homework. confirm your submission by the date-timestamp on the Course page.y\nOlder news:[source1] [source2]', 'section': 'General course-related questions', 'course': 'data-engineering-zoomcamp'}, vector=None, shard_key=None, order_value=None)])

In [31]:
print(search("What if I submit homeworks late?").points[0].payload['text'])

No, late submissions are not allowed. But if the form is still not closed and it’s after the due date, you can still submit the homework. confirm your submission by the date-timestamp on the Course page.y
Older news:[source1] [source2]


In [33]:
client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword" # exact matching on string metadata fields
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [20]:
def search_in_course(query, course="mlops-zoomcamp", limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle
        ),
        query_filter=models.Filter( # filter by course name
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [21]:
print(search_in_course("What if I submit homeworks late?", "mlops-zoomcamp").points[0].payload['text'])

Please choose the closest one to your answer. Also do not post your answer in the course slack channel.
